<h1>Лабораторна робота #2 </h1>
<h2>Наука про дані: підготовчий етап</h2>
<h3>Виконав: Воронюк Володимир ФБ-24</h3>
<p>
  <b>Мета роботи:</b> ознайомитися з основними кроками по роботі з даними –
  workflow від постановки задачі до написання пояснювальної записки,
  зрозуміти постановку задачі та природу даних, над якими виконується
  аналітичні операції
</p>
<p>
  <b>Основні поняття:</b> сирі дані (raw data), підготовка даних (data preparation)
</p>

<h2>Хід виконання роботи</h2>

<p> - Створити env в якому будуть встановлені всі необхідні бібліотеки та
  налаштування для данної лабораторної роботи
</p>
<pre>
$ python -m venv "lab_2"
$ source activate lab_2/bin/activate
$ pip install urllib3 numpy pandas matplotlib ipython
</pre>

<p>
- Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс
(номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;
</p>

In [1]:
import urllib3
import datetime
import pandas
import os

data_dir_path = "dfs"

#Видалити всі старі csv файли
def clear_dir(dir_path):
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print("Failed to delete {}. Reason: {}".format(file_path, e))

#Видалити із фалу всі html теги
def format_data(data, byte_str_list):
    for byte_str in byte_str_list:
        data = data.replace(byte_str, b'')
    return data
    

#Завантажити csv файл для
def get_data_for_province(province_id, country_dir_path, country, start_year=1981, end_year=None):
    curr_year = datetime.datetime.now().year

    if (end_year is None) or (end_year > curr_year) or (end_year < start_year):
        end_year = curr_year

    data_type = "Mean" 
    curr_datetime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

    data_url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php" + \
        "?country={}&provinceID={}&year1={}&year2={}&type={}".format(country, province_id, str(start_year), str(end_year), data_type)
    data_path = "{}/NOAA_ID{}_{}.csv".format(country_dir_path, province_id, curr_datetime)

    responce = urllib3.request("GET", data_url)
    if responce.status != 200:
        print("Can not get data from server. URL {}".format(data_url))
        return

    delete_bstr_list = [b'<tt>', b'</tt>', b'<pre>', b'</pre>', b'<br>']
    formated_data = format_data(responce.data, delete_bstr_list)
    
    with open(data_path, "wb") as raw_data_file:
        raw_data_file.write(formated_data)
    

#Завантажити csv файли для всіх провінцій країни
def get_data_for_counry(country, province_number, dir_path):
    if not (os.path.exists(dir_path) and os.path.isdir(dir_path)):
        print("{} does not exist.")
        return

    country_dir_path = "{}/{}".format(dir_path, country)
    if not (os.path.exists(country_dir_path) and os.path.isdir(country_dir_path)):
        os.mkdir(country_dir_path)
    else:
        clear_dir(country_dir_path)

    for i in range(1, province_number + 1):
        get_data_for_province(i, country_dir_path, country)


In [2]:
get_data_for_counry("UKR", 27, data_dir_path)

<p>
- Зчитати завантажені текстові файли у фрейм (детальніше про роботу
із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для
сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури,
яка на вхід приймає шлях до директорії, в якій зберігаються файли;
</p>

In [22]:
def get_province_name(file_path):

    data_file =  open(file_path)
    data_info = data_file.readline()

    province_name = data_info[data_info.find(":") + 2:data_info.find(",")]
    province_id = data_info[data_info.find("=") + 2:data_info.find(":")]
    data_file.close()
    return province_id, province_name

def get_province_names(dir_path):
    if not (os.path.exists(dir_path) and os.path.isdir(dir_path)):
       print("{} does not exist".format(dir_path))
       return

    province_names = []
    
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        province_id, province_name = get_province_name(file_path)
        province_names.append((int(province_id), province_name))

    return province_names


def create_dataframe_for_province(file_path):
    headers = ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI", "empty"]

    province_id, _ = get_province_name(file_path)
    
    df = pandas.read_csv(file_path, header=1, names=headers)
    df.insert(0, "province_id", province_id, True)
    df = df.drop(columns=["empty"])
    return df

def create_dataframe_for_country(dir_path, country): #, province_num):
    if not (os.path.exists(dir_path) and os.path.isdir(dir_path)):
        print("{} does not exists.".format(dir_path))
        return
    country_dir_path = os.path.join(dir_path, country)

    if not (os.path.exists(country_dir_path) and os.path.isdir(country_dir_path)):
        print("{} does not exists.".format(dir_path))
        return

    province_df_list = []
    for file_name in os.listdir(country_dir_path):
        file_path = os.path.join(country_dir_path, file_name)
        province_df = create_dataframe_for_province(file_path)
        province_df_list.append(province_df)

    country_df = pandas.concat(province_df_list)
    return country_df


In [23]:
df = create_dataframe_for_country(data_dir_path, "UKR")
df.head()

,province_id,year,week,SMN,SMT,VCI,TCI,VHI
0,10,1982,1,0.059,258.24,51.11,48.78,49.95
1,10,1982,2,0.063,261.53,55.89,38.20,47.04
2,10,1982,3,0.063,263.45,57.30,32.69,44.99
3,10,1982,4,0.061,265.10,53.96,28.62,41.29
4,10,1982,5,0.058,266.42,46.87,28.57,37.72


<p>
- Реалізувати процедуру, яка змінить індекси областей, які використані
на порталі NOAA на наступні:
</p>
<pre>
No області/Назва    No області/Назва
1 Вінницька         14 Миколаївська
2 Волинська         15 Одеська
3 Дніпропетровська  16 Полтавська
4 Донецька          17 Рівенська
5 Житомирська       18 Севастополь
6 Закарпатська      19 Сумська
7 Запорізька        20 Тернопільська
8 Івано-Франківська 21 Харківська
9 Київська          22 Херсонська
10 Місто Київ       23 Хмельницька
11 Кіровоградська   24 Черкаська
12 Луганська        25 Чернівецька
13 Львівська        26 Чернігівська
                    27 Республіка Крим
</pre>

In [24]:
province_index_dict = {
    1: 'Vinnytsya',
    2: 'Volyn',
    3: "Dnipropetrovs'k",
    4: "Donets'k",
    5: 'Zhytomyr',
    6: 'Transcarpathia',
    7: 'Zaporizhzhya',
    8: "Ivano-Frankivs'k",
    9: 'Kiev',
    10: 'Kiev City',
    11: 'Kirovohrad',
    12: "Luhans'k",
    13: "L'viv",
    14: 'Mykolayiv',
    15: 'Odessa',
    16: 'Poltava',
    17: 'Rivne',
    18: "Sevastopol'",
    19: 'Sumy',
    20: "Ternopil'",
    21: 'Kharkiv',
    22: 'Kherson',
    23: "Khmel'nyts'kyy",
    24: 'Cherkasy',
    25: 'Chernivtsi',
    26: 'Chernihiv',
    27: 'Crimea'
} 

province_order = {
    1: 24,
    2: 26,
    3: 25,
    4: 27,
    5: 3,
    6: 4,
    7: 8,
    8: 21,
    9: 22,
    10: 23,
    11: 9,
    12: 10,
    13: 11,
    14: 12,
    15: 13,
    16: 14,
    17: 15,
    18: 16,
    19: 17,
    20: 18,
    21: 19,
    22: 20,
    23: 6,
    24: 1,
    25: 2,
    26: 7,
    27: 5
}
def change_province_indeces(df, province_order):
    for old, new in province_order.items():
        df.loc[df["province_id"] == str(old), "province_id"] = new


In [25]:
change_province_indeces(df, province_order)
df.loc[df["province_id"] == 1].head()

,province_id,year,week,SMN,SMT,VCI,TCI,VHI
0,1,1982,1,0.068,263.59,63.47,28.34,45.90
1,1,1982,2,0.074,265.78,67.62,23.05,45.34
2,1,1982,3,0.076,267.19,69.37,20.40,44.88
3,1,1982,4,0.075,268.57,65.26,17.93,41.60
4,1,1982,5,0.072,269.24,58.58,20.00,39.29


<p>
- Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
</p>

<p>
-- Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);
<p>

In [7]:
def get_vhi(df, province_id, year_range):
    df = df.drop(df.loc[df["VHI"] == -1].index)
    vhi_list = df.loc[(df["province_id"] == province_id)\
                       & (df["year"] >= year_range[0])\
                       & (df["year"] <= year_range[1])]["VHI"]

    return vhi_list.tolist(), (vhi_list.min(), vhi_list.max())

def demo_get_vhi():
    province_number = len(df["province_id"].unique())
    year_range = (2000, 2000)
    for i in range(1, province_number + 1):
        vhi_list, vhi_extremes = get_vhi(df, i, year_range)
        print("For province {} in {}-{}:".format(province_index_dict[i], year_range[0], year_range[1]))
        print("VHI: {}".format(vhi_list[0:20]))
        print("Max VHI: {}; Min VHI: {}".format(vhi_extremes[0], vhi_extremes[1]))
        print("-" * 100)


In [8]:
demo_get_vhi()

For province Vinnytsya in 2000-2000:
VHI: [24.22, 27.7, 30.68, 32.55, 34.73, 35.08, 33.79, 34.6, 37.7, 38.67, 38.05, 39.32, 40.21, 41.25, 43.66, 47.61, 52.54, 60.14, 62.91, 63.27]
Max VHI: 11.25; Min VHI: 63.27
----------------------------------------------------------------------------------------------------
For province Volyn in 2000-2000:
VHI: [24.65, 27.49, 31.36, 37.28, 40.85, 41.53, 41.41, 42.27, 44.28, 45.89, 47.46, 50.11, 50.6, 50.12, 49.23, 52.96, 59.41, 65.21, 68.54, 68.85]
Max VHI: 24.65; Min VHI: 68.85
----------------------------------------------------------------------------------------------------
For province Dnipropetrovs'k in 2000-2000:
VHI: [39.02, 42.24, 45.66, 47.38, 48.96, 48.61, 46.32, 45.79, 45.54, 45.75, 44.65, 43.32, 40.39, 36.16, 34.9, 35.48, 39.23, 47.71, 53.21, 53.0]
Max VHI: 17.77; Min VHI: 61.55
----------------------------------------------------------------------------------------------------
For province Donets'k in 2000-2000:
VHI: [33.12, 34.14, 36.

<p>
-- Ряд VHI за вказаний діапазон років для вказаних областей;
</p>

In [28]:
def get_vhi_for_range(df, provinces, year_range):
    df = df.drop(df.loc[df["VHI"] == -1].index)
    vhi_for_range =  df.loc[(df["province_id"].isin(provinces))\
                            & (df["year"] >= year_range[0])\
                            & (df["year"] <= year_range[1])][["province_id", "year", "week", "VHI"]]
        
    return vhi_for_range
            
def demo_get_vhi_for_range():
    provinces = [3, 5, 10, 12]
    year_range = (2000, 2005)

    vhi_range_list = get_vhi_for_range(df, provinces, year_range)
    print("VHI for provinces: ", end="")

    for prov in provinces:
        print("\"{}\" ".format(province_index_dict[prov]), end="")
    print()

    print("Year range: {} - {}".format(year_range[0], year_range[1]))
    print("{}".format(vhi_range_list))



In [29]:
demo_get_vhi_for_range()

VHI for provinces: "Dnipropetrovs'k" "Zhytomyr" "Kiev City" "Luhans'k" 
Year range: 2000 - 2005
     province_id  year  week    VHI
936           10  2000     1  18.21
937           10  2000     2  23.11
938           10  2000     3  25.44
939           10  2000     4  29.79
940           10  2000     5  33.29
...          ...   ...   ...    ...
1243           3  2005    48  40.70
1244           3  2005    49  42.92
1245           3  2005    50  41.13
1246           3  2005    51  41.49
1247           3  2005    52  44.08

[1168 rows x 4 columns]


<p>
-- Виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
</p>
<p>
-- Аналогічно для помірних посух
</p>

In [17]:
def get_dry_years(df, percentage, vhi_range, year_range):
    dry_years = []
    number_of_prov = len(df["province_id"].unique())
    df = df.drop(df.loc[df["VHI"] == -1].index)

    for year in range(year_range[0], year_range[1] + 1):
        number_of_dry_prov = len(df.loc[(df["VHI"] >= vhi_range[0])\
                                        & (df["VHI"] <= vhi_range[1])\
                                        & (df["year"] == year)]["province_id"].unique())

        if (number_of_dry_prov / number_of_prov * 100) >= percentage:
            dry_years.append(year)

    return dry_years

def demo_dry_years(df, year_range, vhi_range, perc):
    dry_years = get_dry_years(df, perc, vhi_range, year_range)
    print("Year range: {}-{}; VHI range: {}-{}; percentage: {}".format(year_range[0], year_range[1], vhi_range[0], vhi_range[1], perc))
    print("Years: {}".format(dry_years))


In [19]:
print("Extreme droughts")
demo_dry_years(df, (1982, 2024), (0, 15), 15)
print('-' * 100)
print("Moderate droughts")
demo_dry_years(df, (1982, 2024), (15, 35), 20)

Extreme droughts
Year range: 1982-2024; VHI range: 0-15; percentage: 15
Years: [2000, 2007]
----------------------------------------------------------------------------------------------------
Moderate droughts
Year range: 1982-2024; VHI range: 15-35; percentage: 20
Years: [1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
